In [1]:
import bw2data as bd
import bw2io as bi
from pathlib import Path

In [2]:
import bw2data as bd
import bw2io as bi

PROJECT_NAME = "linking LCA & dMFA_production"

bi.restore_project_directory(
    fp='/etc/data/ecoinvent-3.10-cutoff-bw25.tar.gz',
    project_name=PROJECT_NAME, overwrite_existing=True
)



Restoring project backup archive - this could take a few minutes...
Restored project: linking LCA & dMFA_production


'linking LCA & dMFA_production'

In [3]:
bd.projects.set_current("linking LCA & dMFA_production")


07:41:01+0000 [info     ] Applying automatic update: 4.0 database search directories FTS5
07:41:01+0000 [info     ] Reindexing database ecoinvent-3.10-biosphere
07:41:01+0000 [info     ] Reindexing database ecoinvent-3.10-cutoff


In [4]:
print(bd.databases)

Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff


In [5]:
db = bd.Database("ecoinvent-3.10-cutoff")
print(len(db))  # Total number of activities
print(list(db)[:3])  # Display a few sample activitiesn

23523
['market for electricity, high voltage' (kilowatt hour, CI, None), 'clinker production' (kilogram, PE, None), 'heat and power co-generation, lignite' (megajoule, PL, None)]


In [6]:
db = bd.Database("ecoinvent-3.10-cutoff")
# Let the metadata system know this database exists. Not necessary if using a `bw2io` importer.
db.register(hi_mom=True, language="Singlish")

In [7]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In [8]:
db.metadata

{'overwrite': False,
 'format': 'Ecoinvent XML',
 'depends': ['ecoinvent-3.10-biosphere'],
 'backend': 'sqlite',
 'number': 0,
 'modified': '2025-06-25T07:41:06.184644',
 'geocollections': [],
 'processed': '2025-06-25T07:41:06.186182',
 'dirty': False,
 'searchable': True}

In [9]:
kettle = db.new_node(
    name="kettle",
    unit="number",
    type=bd.labels.product_node_default,
)

kettle_production = db.new_node(
    name="kettle production",
    location="RER",
    type=bd.labels.process_node_default,
)

kettle.save()
kettle_production.save()

In [10]:
#electricity = db.new_node(
    #name='Electricity',
    #unit='kilowatt hour',
    #type=bd.labels.product_node_default,
#)
#electricity_production = db.new_node(
    #name='electricity production',
    #location='',
    #type=bd.labels.process_node_default,
#)

#electricity.save()
#electricity_production.save()

In [11]:
slt1_production = db.new_node(
    code="slt-production",
    name='stainless steel production', #market for steel, chromium steel 18/8
    location='RER',
    type=bd.labels.process_node_default,
)
slt1 = db.new_node(
    code="slt",
    name='steel',
    unit="kilogram",
    type=bd.labels.product_node_default,
)


slt1_production.save()
slt1.save()

In [12]:
slt2_production = db.new_node(
    code="slt-production2",
    name='stainless steel production2', #market for steel, chromium steel 18/8
    location='RER',
    type=bd.labels.process_node_default,
)
slt2 = db.new_node(
    code="slt2",
    name='steel2',
    unit="kilogram",
    type=bd.labels.product_node_default,
)


slt2_production.save()
slt2.save()

In [13]:
#cp_production = db.new_node(
    #code="cp-production",
    #name='copper production', #market for polypropylene, granulate
    #location='DE',
    #type=bd.labels.process_node_default,
#)
#cp = db.new_node(
    #code="cp",
    #name='copper',
    #unit="kilogram",
    #type=bd.labels.product_node_default,
#)


#cp_production.save()
#cp.save()

In [14]:
co2 = db.new_node(
    name="Carbon Dioxide", 
    context=('air',),
    tags={'CAS Number': '124-38-9'},
    unit='kilogram',
    type=bd.labels.biosphere_node_default,
)

co2.save()

In [39]:
kettle_production.new_edge(
    amount=1,
    input=kettle,
    type=bd.labels.production_edge_default,
).save()
slt1_production.new_edge(
    amount=1,
    input=slt1,
    type=bd.labels.production_edge_default,
).save()
slt2_production.new_edge(
    amount=1,
    input=slt2,
    type=bd.labels.production_edge_default,
).save()
#cp_production.new_edge(
   # amount=1,
    #input=cp,
    #type=bd.labels.production_edge_default,
#).save()
#electricity_production.new_edge(
    #amount=1,
    #input=electricity,
    #type=bd.labels.production_edge_default,
#).save()    

In [40]:
materials = {
    slt1: 0.546246,
    slt2: 0,
    #cp: 0.017046,
    # ... other nodes and their corresponding values in kilograms
}

for material, amount in materials.items():
    kettle_production.new_edge(
        amount=amount,
        type=bd.labels.consumption_edge_default,
        input=material
    ).save()

In [41]:
#slt_production.new_edge(
    #amount=0.01389,  # 0.01389 kWh of electricity, in the papaer (MARCINKOWSKI, 2017)
    #uncertainty_type=5, 
    #minimum=0.012, 
    #maximum=0.016,
    #type=bd.labels.consumption_edge_default,
    #input=electricity,
#).save()



In [42]:
#cp_production.new_edge(
    #amount=0.01389,  
    #uncertainty_type=5, 
    #minimum=0.012,  
    #maximum=0.016,  
    #type=bd.labels.consumption_edge_default,
    #input=electricity,
#).save()

In [43]:
slt1_production.new_edge(
    amount=4.62, 
    uncertainty_type=5, 
    minimum=4.158,
    maximum=5.082, 
    type=bd.labels.biosphere_edge_default,
    input=co2,
).save()

In [44]:
slt2_production.new_edge(
    amount=0, 
    uncertainty_type=5, 
    minimum=0,
    maximum=0, 
    type=bd.labels.biosphere_edge_default,
    input=co2,
).save()

In [45]:
import bw2calc as bc

In [ ]:
functional_unit, data_objs, _ = bd.prepare_lca_inputs({kettle: 1}, remapping=False)

In [ ]:
lca = bc.LCA(demand=functional_unit, data_objs=data_objs)
lca.lci()

In [ ]:
lca.inventory[lca.dicts.biosphere[co2.id], :].sum()